In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = GermanDataset(device=device)

In [3]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_fb_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_fb_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_fb_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_fb_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_fb_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

# SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['delta_effort'] = 0.5

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 155.70epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 170.43epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 162.78epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 160.62epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei        dp          eo        eodd
---------------  ---------  ----------------  --------------  ----------  --------  ----------  ----------
         0.0001          0          0.303125         0.3      0.00282271  0.009375  0.0172208   0.0172208
         0.001           0          0.68125          0.7      0.0333333   0.046875  0.0643387   0.0643387
         0.01            0          0.7375           0.775    0.00952381  0.040625  0.00980627  0.00980627
         0.1             0          0.767188         0.74375  0.183333    0         0.00191342  0.117021


# EI Hyperparameter Selection

In [12]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [1e-3, 5e-3]
EI_hp_test['lambda_'] = [0.001, 0.1, 0.2, 0.5, 0.8]
EI_hp_test['batch_size'] = 128
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 93.72epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:01<00:00, 98.16epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 119.81epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 131.69epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp           eo        eodd
---------------  ---------  ----------------  --------------  ---------  --------  -----------  ----------
          0.001      0.001          0.710938         0.75625  0.0714286  0.021875  0.000478354  0.0159574
          0.001      0.1            0.709375         0.73125  0.291667   0.028125  0.00932791   0.0276596
          0.001      0.2            0.70625          0.70625  1          0.0125    0.00884956   0.0287234
          0.001      0.5            0.7              0.70625  0          0         0            0
          0.001      0.8            0.696875         0.70625  0          0         0            0
          0.005      0.001          0.760938         0.74375  0.138889   0.009375  0.00143506   0.130851
          0.005      0.1            0.765625         0.75     0.231884   0.003125  0.0102846    0.130851
          0.005      0.2     

In [6]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.5

# DP Hyperparameter Selection

In [7]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [1e-3, 1e-2]
DP_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
DP_hp_test['batch_size'] = 128
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 106.89epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 118.24epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 126.37epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 126.75epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  --------  ----------  ---------
          0.001        0            0.714063         0.75     0.0380952   0.015625  0.0083712   0.0159574
          0.001        0.2          0.704688         0.75625  0.114474    0.078125  0.0746233   0.0746233
          0.001        0.5          0.642188         0.71875  0.0323077   0.065625  0.0760584   0.0760584
          0.001        0.8          0.534375         0.5875   0.0184448   0.053125  0.0798852   0.0798852
          0.001        1            0.439063         0.4875   0.0298705   0.003125  0.0105238   0.0404255
          0.001        2            0.28125          0.29375  0.0180545   0.01875   0.0303755   0.103191
          0.001       10            0.25625          0.25625  0.0361905   0.00625   0.0157857   0.0744681
          0.01

# EO Hyperparameter Selection

In [8]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [1e-3, 1e-2]
EO_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EO_hp_test['batch_size'] = 128
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 116.25epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 114.05epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 117.83epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 116.87epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ---------
          0.001        0            0.714063         0.75     0.0380952  0.015625  0.0083712   0.0159574
          0.001        0.2          0.707812         0.7375   0.121212   0.021875  0.0265487   0.0797872
          0.001        0.5          0.709375         0.71875  0.25       0.009375  0.00884956  0.0361702
          0.001        0.8          0.701562         0.70625  0          0         0           0
          0.001        1            0.7              0.70625  0          0         0           0
          0.001        2            0.3              0.29375  0          0         0           0
          0.001       10            0.3              0.29375  0          0         0           0
          0.01         0            0.767188         0

# EOD Hyperparameter Selection

In [9]:
EOD_hp_test = SGD_hp_test.copy()
EOD_hp_test['learning_rate'] = [1e-3, 1e-2]
EOD_hp_test['lambda_'] = [0, 0.2, 0.5, 0.8, 1, 2, 10]
EOD_hp_test['batch_size'] = 128
EOD_hp_test['fairness'] = 'EOD'

hyperparameter_test(dataset, EOD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 154.73epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 153.24epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 150.79epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:00<00:00, 146.20epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ---------
          0.001        0            0.714063         0.75     0.0380952  0.015625  0.0083712   0.0159574
          0.001        0.2          0.714063         0.75     0.0380952  0.015625  0.0083712   0.0159574
          0.001        0.5          0.714063         0.75     0.0380952  0.015625  0.0083712   0.0159574
          0.001        0.8          0.714063         0.75     0.0380952  0.015625  0.0083712   0.0159574
          0.001        1            0.285938         0.29375  0          0.0125    0.00884956  0.0287234
          0.001        2            0.3              0.29375  0          0         0           0
          0.001       10            0.3              0.29375  0          0         0           0
          0.01         0            0.